In [ ]:
# !pip install deepdish
!pip install tf_slim

     |████████████████████████████████| 358kB 2.7MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import sys
sys.path.append('./drive/My Drive/project/code')
sys.path
from function import *
import os
import numpy as np
import matplotlib.pyplot as plt
# import deepdish as dd
import tensorflow as tf
from PIL import Image
from io import BytesIO
import pickle
tf.config.experimental.list_physical_devices('GPU')
tf.config.set_soft_device_placement(True)
input_size=512

In [ ]:
def uncompress(s):
  return np.array(Image.open(BytesIO(s)))
def crop_kinect(im):
  # the bounds of the table, plus some padding above for tall objects
  bounds = np.array([[ 0.28602991, 0.07516428],
                     [ 0.76788474, 0.06441159],
                     [ 0.97554603, 0.59487754],
                     [ 0.13482023, 0.60563023]])
  d = np.array([im.shape[1], im.shape[0]])
  x0, y0 = map(int, bounds.min(0) * d)
  x1, y1 = map(int, bounds.max(0) * d)
  return im[y0 : y1, x0 : x1]

def compress(im, format = 'png'):
  out = BytesIO()
  im  = Image.fromarray(np.uint8(im))
  im.save(out, format = format)
  c = out.getvalue()
  out.close()
  return c

def im(x, crop = False, comp = True):
  x = np.array(x)
  if crop:
    x = crop_kinect(x)
  x = scale(x, (input_size, input_size), 1)
  if comp:
    x = compress(x)
  return x
def save(fname, x):
  if x.__class__.__name__ == 'AsyncMapResult':
    raise RuntimeError('Error: tried to save AsyncMapResult')
  f = open(fname, 'wb')
  pickle.dump(x, f)
  f.close()

def load(fname):
  f = open(fname, 'rb')
  x = pickle.load(f)
  f.close()
  return x

In [ ]:
pr=gel_im_fulldata_v5()

out_dir=pr.dsdir
base_data = './drive/My Drive/project/data'

In [ ]:
data = []
folderIn = './drive/My Drive/project/data/'
db_list=[x for x in range(35,38)]
for l in db_list: 
  print(l)
  if l<10:  
    namefile = 'calandra_corl2017_00%d.h5'%l
  else:
    namefile = 'calandra_corl2017_0%d.h5'%l
  db_file = folderIn+namefile
  # with h5py.File(db_file, 'r') as db_all:
  # db_all = h5py.File(db_file,'r')
  db_all = dd.io.load(db_file)
  n=len(db_all)
  for i in range(n):
    db=db_all[i]
    is_gripping = int(np.array(db['is_gripping']))
    d = dict(gelsightA_before = im(db['gelsightA_before']),
          gelsightB_before = im(db['gelsightB_before']),
          gelsightA_during = im(db['gelsightA_during']),
          gelsightB_during = im(db['gelsightB_during']),
          kinectA_rgb_before = im(db['kinectA_rgb_before'], crop = True),
          kinectA_rgb_during = im(db['kinectA_rgb_during'], crop = True),
          is_gripping = int(is_gripping),
          object_name = str(np.array(db['object_name'])))
    
    data.append(d)

35
36
37


In [ ]:
# data_all+=data
data_all=data.copy()

In [ ]:
db_all=[]
data=[]
len(data_all)

546

In [ ]:
pk_file = os.path.join(out_dir, '%s.pk' % 'train_512')
save(pk_file, data)

In [ ]:
# tf_file = os.path.join(out_dir, '%s.tf' % 'test_34')
writer = tf.compat.v1.python_io.TFRecordWriter(tf_file)
for d in data:
  fbl = lambda x : tf.compat.v1.train.Feature(bytes_list = tf.compat.v1.train.BytesList(value = [x]))
  fl = lambda x : tf.compat.v1.train.Feature(float_list = tf.compat.v1.train.FloatList(value = map(float, x.flatten())))
  il = lambda x : tf.compat.v1.train.Feature(int64_list = tf.compat.v1.train.Int64List(value = x))

  feat = {'gelsightA_before': fbl(d['gelsightA_before']),
          'gelsightB_before': fbl(d['gelsightB_before']),
          'gelsightA_during': fbl(d['gelsightA_during']),
          'gelsightB_during': fbl(d['gelsightB_during']),
          'kinectA_rgb_before': fbl(d['kinectA_rgb_before']),
          'kinectA_rgb_during': fbl(d['kinectA_rgb_during']),
          'is_gripping' : il([d['is_gripping']])}
  ex = tf.compat.v1.train.Example(features = tf.compat.v1.train.Features(feature = feat))
  writer.write(ex.SerializeToString())
writer.close()
# save(pk_file, data_31_33)

In [ ]:

data0_9=load(pj('./drive/My Drive/project/result/', 'train128_set1.pk'))
data10_15=load(pj('./drive/My Drive/project/result/', 'train128_set2.pk'))
data16_22=load(pj('./drive/My Drive/project/result/', 'train128_set3.pk'))
data23_30=load(pj('./drive/My Drive/project/result/', 'train128_set4.pk'))
# data31_33=load(pj('./drive/My Drive/project/result/', 'train_512_35.pk'))
# data34_37=load(pj('./drive/My Drive/project/result/', 'train_512_37.pk'))
data=data0_9+data10_15+data16_22+data23_30
# print(len(data0_9),len(data10_15),len(data16_22),len(data23_30),len(data31_33))
# data=load(pj('./drive/My Drive/project/result/', 'train256_set4.pk'))

In [ ]:
keys=['gelsightA_before', 'gelsightB_before', 'gelsightA_during', 'gelsightB_during', 'kinectA_rgb_before', 'kinectA_rgb_during']
for i in range(len(data)):
  if i%100==0:
    print(i)
  for key in keys:
    data[i][key]=scale(uncompress(data[i][key]),(128, 128), 1)
    data[i][key] = compress(data[i][key])

In [ ]:
uncompress(data[3]['gelsightB_before']).shape

(128, 128, 3)

In [ ]:
# im=uncompress(data[0]['gelsightA_before'])
# im=scale(im, (256, 256), 1)
pk_file = os.path.join(pr.dsdir, '%s.pk' % 'train128_set4')
save(pk_file, data)

In [ ]:
tf_file = os.path.join(pr.dsdir, '%s.tf' % 'train128_set4')
writer = tf.compat.v1.python_io.TFRecordWriter(tf_file)
for d in data:
  fbl = lambda x : tf.compat.v1.train.Feature(bytes_list = tf.compat.v1.train.BytesList(value = [x]))
  fl = lambda x : tf.compat.v1.train.Feature(float_list = tf.compat.v1.train.FloatList(value = map(float, x.flatten())))
  il = lambda x : tf.compat.v1.train.Feature(int64_list = tf.compat.v1.train.Int64List(value = x))

  feat = {'gelsightA_before': fbl(d['gelsightA_before']),
          'gelsightB_before': fbl(d['gelsightB_before']),
          'gelsightA_during': fbl(d['gelsightA_during']),
          'gelsightB_during': fbl(d['gelsightB_during']),
          'kinectA_rgb_before': fbl(d['kinectA_rgb_before']),
          'kinectA_rgb_during': fbl(d['kinectA_rgb_during']),
          'is_gripping' : il([d['is_gripping']])}
  ex = tf.compat.v1.train.Example(features = tf.compat.v1.train.Features(feature = feat))
  writer.write(ex.SerializeToString())
writer.close()

In [ ]:
data = data0_9 + data10_15 + data16_22 + data23_30
# data0_9=[]
# data10_15=[]
# data16_22=[]
# data23_30=[]
# data31_33=[]
# data34_37=[]
print(len(data))

In [ ]:
train_data=load(pj('./drive/My Drive/project/result/', 'train.pk'))
test_data=load(pj('./drive/My Drive/project/result/', 'test.pk'))
data=train_data+test_data

In [ ]:
def add_dict_list(m, k, v):
  if k in m:
    m[k].append(v)
  else:
    m[k] = [v]

In [ ]:
object_dic={}
for i in range(len(data)):
  add_dict_list(object_dic, data[i]['object_name'], i)

In [ ]:
# seed: 1 8 17
key=list(object_dic.keys())
import random
random.seed(20)
random.shuffle(key)

In [ ]:
s=[]
for k in key:
  s.append(len(object_dic[k]))


In [ ]:
s.sort()
print(s)
k=np.array(s)

[2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 7, 7, 8, 9, 9, 10, 10, 10, 10, 11, 12, 12, 12, 13, 15, 15, 16, 16, 17, 19, 19, 20, 21, 27, 27, 28, 31, 33, 34, 34, 36, 38, 38, 39, 39, 40, 40, 42, 43, 45, 48, 52, 54, 56, 58, 58, 60, 60, 63, 64, 67, 72, 73, 75, 76, 76, 79, 80, 80, 82, 84, 87, 92, 94, 100, 101, 107, 109, 122, 124, 127, 128, 142, 157, 163, 172, 182, 192, 194, 216, 224, 240, 251, 313, 350, 354, 520, 558, 709, 772]


In [ ]:
# sum(k>200)
len(data23_30)

2719

In [ ]:
itest_nosee=[]
num_test=26
for i in range(3*num_test,105):
  test_nosee+=object_dic[key[i]]
len(test_nosee)

2719

In [ ]:
train_see=[]
for i in range(3*num_test,105):
  train_see+=object_dic[key[i]]
len(train_see)

2719

In [ ]:
test_nosee_set=[]
for n in test_nosee:
  test_nosee_set.append(data[n])
len(test_nosee_set)
pk_file = os.path.join(pr.dsdir, '%s.pk' % 'train512_set4')
save(pk_file, test_nosee_set)

In [ ]:
random.shuffle (train_see)
train_see_set=[]
for n in train_see:
  train_see_set.append(data[n])
len(train_see_set)

6689

In [ ]:
random.shuffle (train_see_set)
# train_see_data = train_see_set[:6000]
pk_file = os.path.join(pr.dsdir, '%s.pk' % 'train_3')
save(pk_file, train_see_set)

# val_see_data=train_see_set[6000:]
# pk_file = os.path.join(pr.dsdir, '%s.pk' % 'val_see')
# save(pk_file, val_see_data)

In [ ]:
tf_file = os.path.join(pr.dsdir, '%s.tf' % 'train_3')
writer = tf.compat.v1.python_io.TFRecordWriter(tf_file)
for d in train_see_set:
  fbl = lambda x : tf.compat.v1.train.Feature(bytes_list = tf.compat.v1.train.BytesList(value = [x]))
  fl = lambda x : tf.compat.v1.train.Feature(float_list = tf.compat.v1.train.FloatList(value = map(float, x.flatten())))
  il = lambda x : tf.compat.v1.train.Feature(int64_list = tf.compat.v1.train.Int64List(value = x))

  feat = {'gelsightA_before': fbl(d['gelsightA_before']),
          'gelsightB_before': fbl(d['gelsightB_before']),
          'gelsightA_during': fbl(d['gelsightA_during']),
          'gelsightB_during': fbl(d['gelsightB_during']),
          'kinectA_rgb_before': fbl(d['kinectA_rgb_before']),
          'kinectA_rgb_during': fbl(d['kinectA_rgb_during']),
          'is_gripping' : il([d['is_gripping']])}
  ex = tf.compat.v1.train.Example(features = tf.compat.v1.train.Features(feature = feat))
  writer.write(ex.SerializeToString())
writer.close()

In [ ]:
import numpy as np
gel_test=np.array([0.7933, 0.7995, 0.8369, 0.8397])*100
gel_train=np.array([0.8952, 0.8739, 0.8692, 0.8671])*100

gel_im_test=np.array([0.8127, 0.8330, 0.8529, 0.8505])*100
gel_im_train=np.array([0.9280, 0.9028, 0.9195, 0.8934])*100

im_test=np.array([0.6333, 0.6991, 0.6834, 0.6649])*100
im_train=np.array([0.8354, 0.8057, 0.8437, 0.8104])*100

during_test=np.array([0.7392, 0.7847, 0.7902, 0.8106])*100
during_train=np.array([0.9440, 0.9274, 0.8884, 0.9164])*100

gel_512_test=np.array([0.8150, 0.8113, 0.8550, 0.8391])*100
gel_512_train=np.array([0.9023, 0.8812, 0.8884, 0.8667])*100

gel_128_test=np.array([0.7859, 0.7891, 0.8383, 0.8296])*100
gel_128_train=np.array([0.9041, 0.8759, 0.8548, 0.8571])*100